In [2]:

import matplotlib.pyplot as plt
import modules.cosmos_functions as cf
import numpy as np
import pprint
import torch
import torch.nn.functional as FF
import torchvision.transforms as T
import torchvision.transforms.functional as F

from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


In [3]:
# functions to display images
def reverse_normalize(image):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    image = image.clone()
    for i in range(3):
        image[i] = (image[i] * std[i]) + mean[i]
    return image

def show_batch(test_d):
    # Get the first batch of data from the DataLoader
    data_test = next(iter(test_d))

    # Retrieve the first tensor and its corresponding label
    image_test = data_test[0][0]
    label_test = data_test[1][0]

    # Reverse the normalization of the image
    image_test = reverse_normalize(image_test)

    # Convert the image tensor to a NumPy array and transpose the dimensions
    np_image_test = image_test.permute(1, 2, 0).numpy()

    # Display the image
    plt.imshow(np_image_test)
    plt.title(f'{label_test}, {image_test.shape}')
    plt.axis('off')

    # Show the plot
    plt.show()


In [4]:
# set the device

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
x = torch.ones(1, device=device)

print(f"Device is '{device}' Thus a tensor will look like this: {x}")

Device is 'mps' Thus a tensor will look like this: tensor([1.], device='mps:0')


In [5]:
# define the CNN model

class CNN(nn.Module):
    
    #-------------------------------------------------------
    
    def __init__(self, dropout=0):
        self.dropout = dropout
        

        # Because we inherit from Module base class
        super().__init__()
        
        # RGB input, 6 filters, kernel of 5 x 5
        self.conv1 = nn.Conv2d(3, 6, 5)
        
        # Filter is 2 x 2 with a stride of 2 (defined once, used two times)
        self.pool = nn.MaxPool2d(2, 2)
        
        # in_channels = 6 because self.conv1 output has 6 channels
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # Fully connected layer matched on output of conv2 layer
        self.fc1 = nn.Linear(16 * 13 * 13, 120)
        
        # Dropout layer1
        self.dropout1 = nn.Dropout(self.dropout)    

        # Dropout layer2
        self.dropout2 = nn.Dropout(self.dropout)

        # Fully connected layer
        self.fc2 = nn.Linear(120, 84)
        
        # We only have 2 labels
        self.fc3 = nn.Linear(84, 10)
        
    #-------------------------------------------------------
        
    def forward(self, x):
        
        # Convolution with relu layers
        x = self.pool(FF.relu(self.conv1(x)))
        x = self.pool(FF.relu(self.conv2(x)))
        
        # To match the output of the conv2 layer onto the first fully connected layer
        # Like reshape() but makes no copy (reuses underlaying data)
        x = x.view(-1, 16 * 13 * 13)
        
        # Fully connected layers
        x = FF.relu(self.fc1(x))
        x = self.dropout1(x)
        x = FF.relu(self.fc2(x))
        x = self.dropout2(x)
        
        # No activation on final layer 
        return self.fc3(x)


In [6]:
# load the dataset
dataset_path = "../storage/images/apple_disease_classification/Test"
# transform and normalize the data
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = ImageFolder(dataset_path, transform=transform)


dataset.class_to_idx
pp = pprint.PrettyPrinter(indent=1)  # Create a PrettyPrinter instance with an indentation of 1 space
pp.pprint(dataset.class_to_idx)  # Use the pprint method to print the dictionary



{'Blotch_Apple': 0, 'Normal_Apple': 1, 'Rot_Apple': 2, 'Scab_Apple': 3}


In [7]:
# load the testset in the dataloader

test_dataset = dataset

# Define the batch size for the DataLoader
batch_size = 120

# Create the DataLoader to load the dataset in batches
#train_d = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_d = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
#val_d = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# print the shape of the 1st image in the dataset
print(test_dataset[0][0].shape)
print(len(test_dataset))


torch.Size([3, 64, 64])
120


In [ ]:
# # Show the 1st img in the dataset
# show_batch(test_d)
# print(test_dataset[0][0])

In [10]:
imported_model_path = "../storage/data/generated/20230605-160707_pinky_acc.pt"   # test to compare to SQL import
# imported_model_path = "../storage/data/generated/20230605-134750_pinky_acc.pt"  # high accuracy
# imported_model_path = cf.load_pth('20230605_160852_pinky')  # issues; WIP


In [11]:


# Load the model
model = CNN()
model_import_path = imported_model_path
model.load_state_dict(torch.load(model_import_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the test dataset
dataset_path = "../storage/images/apple_disease_classification/Test"
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageFolder(dataset_path, transform=transform)
test_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Define the label dictionary
labels_dict = {
    'Blotch_Apple': 0,
    'Normal_Apple': 1,
    'Rot_Apple': 2,
    'Scab_Apple': 3
}

def test_model(model, dataloader):
    model.eval()


    # Track the overall test accuracy and accuracy by each type of apple
    overall_correct = 0
    overall_total = 0
    class_correct = {label: 0 for label in labels_dict.keys()}
    class_total = {label: 0 for label in labels_dict.keys()}

    # Iterate over the test dataset
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)

        # # Calculate loss
        # loss = criterion(outputs, labels)

        # Get predictions
        _, predicted = torch.max(outputs.data, 1)

        # Update accuracy counts
        overall_correct += (predicted == labels).sum().item()
        overall_total += labels.size(0)

        for label, index in labels_dict.items():
            mask = labels == index
            class_predicted = predicted[mask]
            class_labels = labels[mask]
            class_correct[label] += (class_predicted == class_labels).sum().item()
            class_total[label] += class_labels.size(0)

    # Calculate overall accuracy
    overall_accuracy = overall_correct / overall_total

    # Print overall accuracy
    print(f"Overall accuracy: {overall_accuracy:.4f}")

    # Print accuracy by each type of apple
    for label in labels_dict.keys():
        class_accuracy = class_correct[label] / class_total[label]
        print(f"{label} accuracy: {class_accuracy:.4f}")

test_model(model, test_dataloader)


Overall accuracy: 0.3417
Blotch_Apple accuracy: 0.0000
Normal_Apple accuracy: 0.0000
Rot_Apple accuracy: 0.7105
Scab_Apple accuracy: 0.5000
